In [ ]:
def argmax(a, f):
  max_i = 0
  max_v = a[0]
  for i in range(len(a)):
    v = f(i)
    if max_v < v:
      max_v = v
      max_i = i
  return max_i


In [ ]:
import math

import numpy as np
from typing import List
from random import random
from random import randrange

class Policy(object):
  def choose(self, step: int, value_estimates: List, action_attempts: List) -> int:
    return randrange(len(value_estimates))

  def __str__(self) -> str:
    return "Random"


class GreedyPolicy(Policy):
  def __init__(self):
    pass

  def choose(self, step: int, value_estimates: List, action_attempts: List) -> int:
    return argmax(value_estimates, lambda i: value_estimates[i])

  def __str__(self) -> str:
    return "Greedy"


class EpsilonGreedyPolicy(Policy):
  def __init__(self, e: float):
    self.e = e

  def choose(self, step: int, value_estimates: List, action_attempts: List) -> int:
    if random() < self.e:
      return randrange(len(value_estimates))

    return argmax(value_estimates, lambda i: value_estimates[i])

  def __str__(self) -> str:
    return "EpsilonGreedy-{}".format(self.e)

def ucb_selection(value_estimates: List, action_attempts: List, i: int, c: float, step: int):
  if action_attempts[i] == 0:
    return value_estimates[i]
  return value_estimates[i] + math.pow(math.log(step) / action_attempts[i], 1/c)

class UcbPolicy(Policy):
  def __init__(self, e: float, c: float):
    self.e = e
    self.c = c

  def choose(self, step: int, value_estimates: List, action_attempts: List) -> int:
    if random() < self.e:
      if step == 0:
        return randrange(len(value_estimates))
      return argmax(action_attempts, lambda i: ucb_selection(value_estimates, action_attempts, i, self.c, step))

    return argmax(value_estimates, lambda i: value_estimates[i])

  def __str__(self) -> str:
    return "UCB-{}-{}".format(self.e, self.c)


class Agent:
  def __init__(self, k: int, policy: Policy):
    self.policy = policy
    self.value_estimates = []
    self.action_attempts = []
    self.last_action = -1

    for i in range(k):
      self.value_estimates.append(0)
      self.action_attempts.append(0)

  def choose(self, step: int) -> int:
    self.last_action = self.policy.choose(
      step, self.value_estimates, self.action_attempts)
    return self.last_action

  def observe(self, reward: float):
    self.action_attempts[self.last_action] += 1
    regret = reward - self.value_estimates[self.last_action]
    step_size = 1 / float(self.action_attempts[self.last_action])
    self.value_estimates[self.last_action] += regret * step_size


class GaussianBandit:
  def __init__(self, mean: float, variance: float, k: int):
    self.arms = np.random.normal(mean, variance, k)
    self.variance = variance

  def pull(self, arm: int) -> float:
    return np.random.normal(self.arms[arm], self.variance)

  def __str__(self):
    return "{}".format(self.arms)


In [ ]:
bandit = GaussianBandit(0, 1, 10)
print(bandit)

In [ ]:
import pandas as pd

policies = [
  Policy(),
  GreedyPolicy(),
  EpsilonGreedyPolicy(0.2),
  EpsilonGreedyPolicy(0.1),
  EpsilonGreedyPolicy(0.01),
  UcbPolicy(0.01, 2),
]

agents = [Agent(len(bandit.arms), p) for p in policies]

global_total_rewards = []
global_avg_rewards = []
for n in range(1, 1100):
  total_rewards = []
  avg_rewards = []
  for _ in range(len(agents)):
    total_rewards.append(0)
    avg_rewards.append(0)

  for step in range(n):
    for i, agent in enumerate(agents):
      arm = agent.choose(step)
      reward = bandit.pull(arm)
      total_rewards[i] += reward
      avg_rewards[i] = (avg_rewards[i] * step + reward) / (step + 1)
      agent.observe(reward)

  global_total_rewards.append(total_rewards)
  global_avg_rewards.append(avg_rewards)

policy_names = [str(p) for p in policies]
total_rewards_df = pd.DataFrame(data=global_total_rewards, columns=policy_names)
avg_rewards_df = pd.DataFrame(data=global_avg_rewards, columns=policy_names)

display(total_rewards_df)
display(avg_rewards_df)


In [ ]:
import plotly.express as px

px.line(total_rewards_df, labels={"index": "Steps", "value": "Total Reward"}).show()
px.line(avg_rewards_df, labels={"index": "Steps", "value": "Average Reward"}).show()
